<a href="https://colab.research.google.com/github/ArasyH/ArasyH.github.io/blob/main/agentic_patterns_challenge_arasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import json
import asyncio
import requests

from typing import List
from agents import Agent, Runner, function_tool, trace

from google.colab import userdata

SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

headers = {"Authorization": SECTORS_API_KEY}

In [ ]:
def retrieve_from_endpoint(url: str) -> dict:

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

In [ ]:
@function_tool
def get_company_overview(ticker: str, country: str) -> str | None:
    """
    Get company overview from Singapore Exchange (SGX) or Indonesia Exchange (IDX)
    """
    assert country.lower() in ["indonesia", "singapore", "malaysia"], "Country must be either Indonesia, Singapore, or Malaysia"

    if(country.lower() == "indonesia"):
        url = f"https://api.sectors.app/v1/company/report/{ticker}/?sections=overview"
    if(country.lower() == "singapore"):
        url = f"https://api.sectors.app/v1/sgx/company/report/{ticker}/"
    if(country.lower() == "malaysia"):
        url = f"https://api.sectors.app/v1/klse/company/report/{ticker}/"

    try:
        return retrieve_from_endpoint(url)
    except Exception as e:
        print(f"Error occurred: {e}")
        return None


@function_tool
def get_top_companies_ranked(dimension: str) -> List[str]:
    """
    Return a list of top companies (symbol) based on certain dimension (dividend yield, total dividend, revenue, earnings, market_cap, PB ratio, PE ratio, or PS ratio)
    """

    url = f"https://api.sectors.app/v1/companies/top/?classifications={dimension}&n_stock=3"

    return retrieve_from_endpoint(url)

In [ ]:
company_research_agent = Agent(
    name="company_research_agent",
    instructions="Research the company by using the right tool",
    tools=[get_company_overview],
    output_type=str
)


async def main():
  """
  CHALLENGE:
  Modify the code here so that you're not hardcoding one stock ticker at a time
  (who have time for that?!) Instead, use a Multi-Agent workflow to have another
  Agent generate a list of companies that you want to research, based on some
  screening condition, then delegate to the second Agent to do the work
  """
  company_overview = await Runner.run(
    company_research_agent, "BBCA"
  )
  print(company_overview.final_output)

await main()

### PT Bank Central Asia Tbk. (BBCA.JK)

- **Industry:** Banks
- **Sector:** Financials
- **Market Cap:** IDR 1,192,675,910,877,184
- **Market Cap Rank:** 1

#### Address
Menara BCA, Grand Indonesia  
Jalan MH Thamrin No. 1  
Jakarta 10310

#### Contact
- **Phone:** 021-23588000
- **Email:** [investor_relations@bca.co.id](mailto:investor_relations@bca.co.id)
- **Website:** [www.bca.co.id](http://www.bca.co.id)

#### Employee Information
- **Number of Employees:** 24,685

#### Listing Information
- **Listing Board:** Main
- **Listing Date:** 2000-05-31

#### Stock Information
- **Last Close Price:** IDR 9675 (as of 2025-05-23)
- **Daily Price Change:** 0.26%

#### Highs and Lows
- **YTD High:** IDR 9925 (2025-01-03)
- **YTD Low:** IDR 7275 (2025-04-08)
- **All-Time High:** IDR 10950 (2024-09-23)
- **All-Time Low:** IDR 175 (2004-06-08)

#### ESG Score
- **ESG Score:** 21.5

PT Bank Central Asia Tbk. is the largest bank in Indonesia in terms of market capitalization and is part of the Ma